In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def get_Hardy_Weinberg_freq_AB(df_ab, country):
    ab  = df_ab[country][df_ab[country]>0].reset_index()    
    ab_freq = pd.DataFrame([(a1,a2,b1,b2,f1*f2)
                            for (a1,b1,f1) in ab.values
                            for (a2,b2,f2) in ab.values],
                           columns = 'A_1 A_2 B_1 B_2 f'.split())
    return ab_freq

In [ ]:
df = pd.read_excel("data/A-B-DRB1.xlsx")
df['A'] = df.A.apply(lambda x : x.split(":")[0])
df['B'] = df.B.apply(lambda x : x.split(":")[0])
df['DRB1'] = df.DRB1.apply(lambda x : x.split(":")[0])

df_ab = df.groupby("A B".split()).sum()

In [ ]:
pops={}
pops['kor'] = get_Hardy_Weinberg_freq_AB(df_ab, "KORI_freq")
pops['api'] = get_Hardy_Weinberg_freq_AB(df_ab, "API_freq")

list_name = 'list_API_Top50'
genes = pd.read_csv(f"data/{list_name}.txt", header=None, names=['raw'])
genes['A']=genes.raw.apply(lambda x: x[:4])
genes['B']=genes.raw.apply(lambda x: x[4:])
genes=genes.drop_duplicates('raw', keep='first')

In [ ]:
for country, pop in pops.items():
    matches={}
    # coverages = {}
    for idx, gene in genes.iterrows():
        covered = pop.eval('(A_1=="{A}" or A_2=="{A}") and (B_1=="{B}" or B_2=="{B}")'.format(A=gene.A, B=gene.B))
        matches[gene.raw]=covered
#         coverages[gene.raw]=pop[covered].f.sum()

    coverages=pd.DataFrame([(name, pop[covered].f.sum()) for name, covered in matches.items()],
                           columns=['gene', 'coverage']).sort_values('coverage', ascending=False)

    tot=np.repeat(False, matches[coverages.gene[0]].shape)
    accums=[]
    for gene in coverages.gene:
        tot=tot|matches[gene] 
        accums.append((gene, pop[tot].f.sum()))
    accums=pd.DataFrame(accums, columns=['gene', 'coverage_accum'])
#     accums = accums.merge(genes[['raw','source']], how='left', left_on='gene', right_on='raw').drop('raw',axis=1)
    
    coverages.to_csv(f"{list_name}.%s.coverage.csv"%country, index=False)
    accums.to_csv(f"{list_name}.%s.accum.csv"%country, index=False)